In [ ]:
from beamtest_analysis_helper import DecodeBinary
from pathlib import Path
from natsort import natsorted
from tqdm.notebook import tqdm

In [ ]:
path = Path('test/files')
outdir = Path('test/feathers')

if not outdir.exists():
    outdir.mkdir(exist_ok=True)

files = natsorted(list(path.glob('TDC*.bin')))
decoder = DecodeBinary(
    firmware_key=0b0001,
    board_id=[0x17f0f, 0x17f0f, 0x17f0f, 0x17f0f],
    file_list=files[:1],
    save_nem = None,
    skip_filler = True,
)
df, _ = decoder.decode_files()
df.to_feather(outdir / f'test.feather')
del df

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_feather('test/feathers/test.feather')

In [ ]:
test_df = df.loc[df['evt'] == 0]

In [ ]:
test_df

In [ ]:
# Group the DataFrame by 'col,' 'row,' and 'board,' and count the number of hits in each group
hits_count_by_col_row_board = test_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()

# Rename the 'evt' column to 'hits'
hits_count_by_col_row_board = hits_count_by_col_row_board.rename(columns={'evt': 'hits'})

pivot_table = hits_count_by_col_row_board[hits_count_by_col_row_board['board'] == 0].pivot_table(
            index='row',
            columns='col',
            values='hits',
            fill_value=0  # Fill missing values with 0 (if any)
)

In [ ]:
pivot_table